## Callback
- 케라스 콜백 - 모델의 학습 방향, 저장 시점, 학습 정지 시점 등에 대한 상황을 모니터링


1) ModelCheckpoint
- 지정한 평가지표를 기준으로 가장 뛰어난 성능을 보여주는 모델을 저장할 때 사용
- `ModelCheckpoint(filepath, monitor = 'val_loss', verbose = 0, save_best_only = False, save_weights_only = False, mode = 'auto')`
    - filepath: 모델의 저장 경로를 지정
    - monitor: 모니터링할 평가지표를 설정. 모델이 포함하고 있는 모든 지표를 사용할 수 있다.
    - verbose: 콜백의 수행 과정 노출 여부를 지정.
        - 0: 아무런 표시 하지 않음(기본값)
        - 1: progress bar로 표시
        - 2: 매 에폭마다 수행과정을 자세하게 출력함
    - save_best_only: True인 경우, 가장 성능이 뛰어난 모델만 저장. 그보다 좋지 않은 모델의 경우는 덮어쓰지 않는다.
    - save_weights_only: Ture인 경우 모델의 가중치만 저장
    - mode: ['auto', min', 'max'] 중 하나를 사용
        - monitor에서 지정한 평가지표를 기준으로 작동
            - 평가지표가 val_acc인 경우 max를 선택해야 함.
            - 평가지표가 val_loss인 경우 min을 선택해야 함.
            - auto인 경우 평가지표의 이름을 통해 자동으로 유추.


2) EarlyStopping
- 모델 학습 시에 지정된 기간동안 모니터링하는 평가지표에서 성능 향상이 일어나지 않은 경우 학습을 중단
- `EarlyStopping(monitor = 'val_loss', patience = 0, verbose = 0, mode = 'auto')`
    - monitor, verbose, mode는 1)의 ModelCheckpoint와 같다.
    - patience: 지정한 수만큼의 기간에서 평가지표의 향상이 일어나지 않을 경우 학습을 중단(기간 = epoch)
        - 예를 들어 patience = 5일 때, 5에폭동안 성능 향상이 일어나지 않으면 학습을 중단

3) ReduceLROnPlateau
- EarlyStopping 콜백과 같이 patience 인자를 지정하여 지정된 기간동안 평가지표에서 성능 향상이 일어나지 않으면 학습률을 조정하는 콜백
- `ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 10, verbose = 0, min_lr = 0)`
    - monitor, verbose는 1)의 ModelCheckpoint와 같다.
    - factor: 학습률 조정에 사용되는 값(새로운 학습률 = factor * 기존 학습률)
    - min_lr: 학습률의 하한을 지정

4) TensorBoard
- 학습 과정을 편리하게 모니터링할 수 있도록 텐서플로우에서 제공하고 있는 도구
- `TensorBoard(log_dir = './logs', histogram_freq = 0, batch_size = 32, write_graph = True, write_images = True)`
    - log_dir: 텐서보드를 사용할 로그 파일의 저장 경로
    - histogram_freq: 활성화 및 가중치를 히스토그램으로 얼마나 자주 나타낼지를 결정
        - 0인 경우 히스토그램을 생성하지 않는다.
        - 이를 시행하기 위해서는 학습시에 검증 데이터가 전달되어야 한다.
    - batch_size: 히스토그램을 계산하기 위한 배치 크기
    - write_graph: 그래프 시각화에 대한 여부. 기본값은 True
    - write_images: 텐서보드에서 이미지로 시각화하기 위한 가중치의 기록 여부

## Callback함수 사용 방법

1) 콜백 정의

- callbacks = [ 콜백1, 콜백2, ..., 콜백n ]

2) 학습을 위한 fit()함수에서 callbacks 파라미터에 미리 정의한 리스트를 지정

- 
```python
callbacks = [TensorBoard(log_dir = logdir, histogram_freq = 1, write_graph = True, write_images = True),
             EarlyStopping(monitor = 'val_loss', patience = 3, verbose = 1),
             ModelCheckpoint(filpath = filepath, monitor = 'var_loss', verbose = 1, save_best_only = True)
            ]
model.fit(x_train, y_train, batch_size = 32, validation_data = (x_val, y_val), epochs = 10, callbacks = callbacks)
```